<a href="https://colab.research.google.com/github/RJ-Stony/Item-based-CF-dot-production/blob/main/Item_based_%E1%84%92%E1%85%A7%E1%86%B8%E1%84%8B%E1%85%A5%E1%86%B8%E1%84%91%E1%85%B5%E1%86%AF%E1%84%90%E1%85%A5%E1%84%85%E1%85%B5%E1%86%BC_%EC%B5%9C%EC%A2%85_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collaborative Filtering

Memory-Based Algorithm
- **Item based: dot production 없는 버전 구현해볼 것**
  - 내적 안쓰는 Item based 방식 사용 많이 됨
- User based


Model-Based Algorithm
- Latent Factor 협업 필터링 방법 (Matrix Factorization)

# 구글드라이브 연결

파일 > 드라이브 마운트 클릭

In [ ]:
import pandas as pd
import numpy as np

# 데이터 로드

In [ ]:
ratings = pd.read_csv("data\\Movielens\\ratings_refined.csv", usecols=['userId', 'movieId', 'rating'])
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100784,610,166534,4.0
100785,610,168248,5.0
100786,610,168250,5.0
100787,610,168252,5.0


In [ ]:
movies = pd.read_csv("data\\Movielens\\movies_refined.csv", usecols=['movieId', 'title'])
movies

,movieId,title
0,1,Toy Story
1,2,Jumanji
2,3,Grumpier Old Men
3,4,Waiting to Exhale
4,5,Father of the Bride Part II
...,...,...
9698,193581,Black Butler: Book of the Atlantic
9699,193583,No Game No Life: Zero
9700,193585,Flint
9701,193587,Bungo Stray Dogs: Dead Apple


In [ ]:
df = pd.merge(ratings, movies, on='movieId', how='left')
df

,userId,movieId,rating,title
0,1,1,4.0,Toy Story
1,1,3,4.0,Grumpier Old Men
2,1,6,4.0,Heat
3,1,47,5.0,Seven (a.k.a. Se7en)
4,1,50,5.0,"Usual Suspects, The"
...,...,...,...,...
100784,610,166534,4.0,Split
100785,610,168248,5.0,John Wick: Chapter Two
100786,610,168250,5.0,Get Out
100787,610,168252,5.0,Logan


null 값 체크

In [ ]:
df.columns[df.isna().any()].tolist()

[]

영화명 결측치 체크

In [ ]:
df[df['title'].isnull()]

,userId,movieId,rating,title


# 영화 유사도 행렬 준비

유저X 입력되었을 때,

유저X가 본 영화 중 하나인 A 선택

이 영화A와 유사한 영화 찾을 때

사용할 행렬

In [ ]:
movie_user = df.pivot_table(values='rating', index='title', columns='userId')
movie_user # 인덱스가 영화 컬럼이 사람

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.5,NaN,NaN
xXx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,2.0
xXx: State of the Union,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5


## 결측치 처리

null값이 있으면 cosine similarity 함수가 안돌아감

하지만, null값을 0으로 치환하고 계산할경우 결과가 달라짐

(마치 해당 영화를 보고 0점을 준것으로 계산)

# Item-based CF

영화 유사도 기반 추천

(User-based CF는 사용자 유사도 기반 추천)

유저X 입력되었을 때,

유저X가 본 영화 중 하나인 A 선택

이 영화A와 유사한 영화 찾아서 추천

In [ ]:
# + dot production까지 해서 성능을 높일 수 있다. 유저 한명에 대해 예측값들 따라서 for뮨에서 user로 돌면 됨
# 프로젝트는 아이템 기반에 dot production 추가하는 것이 중심 + 더 좋은 전처리 추가하고 병렬 처리로 효율성 증가는 추가 점수
# 발표시 슬라이드는 간단하게 주요 내용만 , 코랩 파일을 통해 코드를 보며 추가 발표함, 실제 추전한거나 전처리 내용 모델구현 등등

# fullmatrix 결측치, 0이 아닌 값 대치
+ 1. 0으로 채우고 유사도 기반 측정, 유사도가 높은 상위 n개로 0값을 다시 대치
+ 2. knn imputer 사용 -> 결측값은 무시하고 존재하는 값만으로 거리를 구해서 knn 대치
+ 3. truncated svd 로 차원 축소

# SVD - surprise + 교차검증  + standard scaler + 유클리디안 유사도

# SVD 하이퍼 파리미터 튜닝

In [ ]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import GridSearchCV
from surprise import accuracy

# 1) 데이터 준비
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'title', 'rating']], reader)

# 2) 파라미터 그리드 정의
param_grid = {
    'n_factors': [20, 50, 100, 150, 200],
    'n_epochs': [10, 20],
    'reg_all': [0.02, 0.05, 0.1]
}

# 3) GridSearchCV 실행
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gs.fit(data)

# 4) 최적 파라미터, 스코어
print("Best RMSE score:", gs.best_score['rmse'])
print("Best params:", gs.best_params['rmse'])

Best RMSE score: 0.8706970491792619
Best params: {'n_factors': 50, 'n_epochs': 20, 'reg_all': 0.05}


# 결측치 채우기

In [ ]:
best_params = gs.best_params['rmse']

# 5) 최적 파라미터로 SVD 모델 '재생성' 후 '재학습'
algo = SVD(**best_params,random_state= 42)
trainset = data.build_full_trainset()  # 데이터 전체로 학습
algo.fit(trainset)

# 6) 결측치 채우기
movie_user_filled = movie_user.copy()
missing_mask = movie_user.isna()

for item_idx, user_idx in zip(*np.where(missing_mask.values)):
    item = movie_user.index[item_idx]    # 'title'
    user = movie_user.columns[user_idx]  # 'userId'
    pred = algo.predict(user, item)
    movie_user_filled.iloc[item_idx, user_idx] = pred.est

movie_user_filled

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,4.303317,3.517324,2.774313,3.402280,3.446326,3.531949,3.085155,3.611506,3.240741,3.157084,...,4.008494,3.329261,3.248544,3.541455,3.189906,3.492944,3.792947,3.508160,3.194234,4.000000
'Hellboy': The Seeds of Creation,4.413513,3.636614,2.421264,3.402820,3.516294,3.693536,2.985950,3.624113,3.223472,3.322402,...,3.985501,3.340471,3.367978,3.638777,3.328787,3.605436,3.809890,3.578641,3.332925,3.738884
'Round Midnight,4.328005,3.502421,2.354929,3.477001,3.409197,3.547083,3.038073,3.343968,3.171455,3.227851,...,3.942012,3.385917,3.237931,3.578935,3.231507,3.414330,3.569006,3.370995,3.083764,3.566304
'Salem's Lot,4.291835,3.603620,2.681605,3.283702,3.439635,3.830161,3.218199,3.627284,3.307425,3.239891,...,4.005174,3.496957,3.321260,3.759696,3.336215,3.605640,3.708424,3.735990,3.397892,3.683063
'Til There Was You,4.315723,3.688600,2.728653,3.240588,3.454965,3.678613,3.238081,3.563800,3.417447,3.268066,...,4.024219,3.479708,3.377999,3.719412,3.383639,3.520539,3.682891,3.550210,3.296704,3.696930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,4.595364,3.912140,2.888851,3.472127,3.784293,3.900082,3.146679,3.778942,3.600723,3.350675,...,4.310144,3.681011,5.000000,3.942885,3.558108,3.803129,4.038410,4.500000,3.444879,3.880257
xXx,3.677118,2.890609,1.629683,2.902047,2.970685,3.032482,2.542399,2.913392,1.000000,2.685540,...,3.351246,2.669628,2.764315,2.962332,2.620812,2.883339,3.068928,3.500000,2.683911,2.000000
xXx: State of the Union,3.723843,3.042548,1.878988,2.794571,2.976939,3.114181,2.599776,2.900240,2.718626,2.593016,...,3.420300,2.889079,2.676790,3.042804,2.699365,2.889371,3.054080,3.112715,2.670390,1.500000


# 스케일링 및 아이템간 유클리디안 유사도 측정

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances

scaler = StandardScaler()
movie_user_mean_scaled = scaler.fit_transform(movie_user_filled)

def compute_euclidean_similarity(rating_matrix):
    """
    유클리디안 거리 기반 유사도를 계산
    """
    # 유클리디안 거리 계산
    distance_matrix = euclidean_distances(rating_matrix)  # 아이템 기반: 전치 필요
    # 유사도로 변환 (거리가 작을수록 유사도가 높음)
    similarity_matrix = 1 / (1 + distance_matrix)  # 거리를 반전하여 유사도로 변환
    return pd.DataFrame(similarity_matrix, index=movie_user.index, columns=movie_user.index)


euclidean_similarity = compute_euclidean_similarity(movie_user_mean_scaled)
euclidean_similarity

title,'71,'Hellboy': The Seeds of Creation,'Round Midnight,'Salem's Lot,'Til There Was You,'Tis the Season for Love,"'burbs, The",'night Mother,(500) Days of Summer,*batteries not included,...,Zulu,[REC],[REC]²,[REC]³ 3 Génesis,anohana: The Flower We Saw That Day - The Movie,eXistenZ,xXx,xXx: State of the Union,¡Three Amigos!,À nous la liberté (Freedom for Us)
title,,,,,,,,,,,,,,,,,,,,,
'71,1.000000,0.085705,0.085780,0.068471,0.078103,0.047433,0.042778,0.083212,0.028896,0.078992,...,0.073518,0.043414,0.090870,0.061536,0.068341,0.031141,0.016564,0.018276,0.032937,0.031006
'Hellboy': The Seeds of Creation,0.085705,1.000000,0.064825,0.077541,0.080965,0.041857,0.038150,0.071636,0.031407,0.087468,...,0.083700,0.047170,0.081767,0.051718,0.060628,0.034313,0.015787,0.017172,0.030564,0.027859
'Round Midnight,0.085780,0.064825,1.000000,0.056581,0.063799,0.064258,0.051705,0.099987,0.025852,0.062371,...,0.056820,0.035027,0.078764,0.086105,0.088047,0.027239,0.018338,0.020477,0.040674,0.036339
'Salem's Lot,0.068471,0.077541,0.056581,1.000000,0.107433,0.035889,0.033719,0.056805,0.033455,0.084746,...,0.075486,0.054350,0.071456,0.044282,0.049808,0.038523,0.014858,0.016141,0.027844,0.025219
'Til There Was You,0.078103,0.080965,0.063799,0.107433,1.000000,0.039205,0.035977,0.061226,0.031947,0.086412,...,0.074597,0.050450,0.078197,0.048791,0.055740,0.036124,0.015288,0.016670,0.029172,0.026677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,0.031141,0.034313,0.027239,0.038523,0.036124,0.021206,0.021046,0.027714,0.036023,0.037163,...,0.035347,0.049886,0.031065,0.023415,0.026080,1.000000,0.011562,0.012285,0.018173,0.016991
xXx,0.016564,0.015787,0.018338,0.014858,0.015288,0.022371,0.021375,0.017970,0.011485,0.015211,...,0.015233,0.012567,0.016788,0.020735,0.018839,0.011562,1.000000,0.061191,0.024234,0.028189
xXx: State of the Union,0.018276,0.017172,0.020477,0.016141,0.016670,0.025706,0.024320,0.020055,0.012140,0.016572,...,0.016534,0.013483,0.018487,0.023637,0.021099,0.012285,0.061191,1.000000,0.027688,0.032569


# 병렬 처리 + 유사한 아이템 n개 기반 CF 모델

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from joblib import Parallel, delayed

def process_user(user, similarity_matrix, data, all_movies, n_movies, top_n):
    watched_title = data[data['userId'] == user]['title']  # 유저가 본 영화

    # 영화 간 유사도
    sub_sim_mat = similarity_matrix.loc[watched_title]
    sub_sim_mat = sub_sim_mat.T.to_numpy()

    #자기 자신 제외
    np.fill_diagonal(sub_sim_mat, 0)

    # 상위 n개의 유사도만 반영
    top_indices = np.argsort(-sub_sim_mat, axis=0)[:top_n]  # 유사도가 높은 상위 n개의 인덱스
    filtered_sim_mat = np.zeros_like(sub_sim_mat)
    for col_idx in range(sub_sim_mat.shape[1]):
        filtered_sim_mat[top_indices[:, col_idx], col_idx] = sub_sim_mat[top_indices[:, col_idx], col_idx]

    sim_N = np.sum(filtered_sim_mat, axis=1) + 1e-10

    # 평점 예측
    watched_user_y = data[data['userId'] == user]['rating'].tolist()
    watched_user_y = np.array(watched_user_y).reshape(-1, 1)
    pred_y = np.matmul(filtered_sim_mat, watched_user_y).flatten() / sim_N

    title_list = [user] * n_movies
    cur_pred = pd.DataFrame(zip(title_list, all_movies, pred_y),
                            columns=['userId', 'title', 'pred_rating'])
    return cur_pred

def modeling_parallel(similarity_matrix, data, top_n, n_jobs): # 5개
    df_pred_all = pd.DataFrame()
    userids = sorted(data['userId'].unique())
    all_movies = similarity_matrix.index
    n_movies = len(all_movies)

    # 병렬 처리로 각 유저별 작업 수행
    results = Parallel(n_jobs=n_jobs)(
        delayed(process_user)(user, similarity_matrix, data, all_movies, n_movies, top_n)
        for user in tqdm(userids)
    )

    # 결과 합치기
    df_pred_all = pd.concat(results, axis=0)
    return df_pred_all

# 성능 측정 및 추천

In [ ]:
%%time
df_pred_all_parallel =modeling_parallel(euclidean_similarity,df,5,-1)
df_pred_all_parallel

100%|██████████| 610/610 [00:32<00:00, 18.99it/s]


CPU times: total: 15.8 s
Wall time: 39.1 s


,userId,title,pred_rating
0,1,'71,0.000000
1,1,'Hellboy': The Seeds of Creation,0.000000
2,1,'Round Midnight,0.000000
3,1,'Salem's Lot,0.000000
4,1,'Til There Was You,0.000000
...,...,...,...
9408,610,eXistenZ,0.000000
9409,610,xXx,2.000000
9410,610,xXx: State of the Union,1.587319
9411,610,¡Three Amigos!,0.000000


# TEST 데이터셋에 대한 성능 확인

In [ ]:
from sklearn.model_selection import train_test_split

_, test_data = train_test_split(df[['userId', 'title', 'rating']],
                test_size=20000, random_state=1234, stratify=df['userId'])
test_data = pd.merge(test_data, df_pred_all_parallel, on=['userId', 'title'], how='left')
test_data

,userId,title,rating,pred_rating
0,464,Escape from L.A.,3.0,3.000000
1,590,Carlito's Way,4.0,4.000000
2,135,"Nightmare Before Christmas, The",4.0,4.000000
3,288,"Naked Gun 2 1/2: The Smell of Fear, The",4.0,4.000000
4,258,Guardians of the Galaxy 2,3.5,3.500000
...,...,...,...,...
19995,599,Thor: Ragnarok,3.0,3.000000
19996,414,Revenge of the Nerds II: Nerds in Paradise,2.0,2.076715
19997,448,"Wild Bunch, The",5.0,5.000000
19998,263,Gosford Park,4.5,4.500000


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

true_y = np.array(test_data['rating'])
pred_y = np.array(test_data['pred_rating'])

mae = mean_absolute_error(y_true=true_y, y_pred=pred_y)
mse = mean_squared_error(y_true=true_y, y_pred=pred_y)
rmse = np.sqrt(mse)

print(f"MAE  : {str(round(mae, 6))}")
print(f"MSE  : {str(round(mse, 6))}")
print(f"RMSE : {str(round(rmse, 6))}")

MAE  : 0.009168
MSE  : 0.004975
RMSE : 0.070537


# 사용자가 안본 영화 중 예측 평점 높은 10개 추천

In [ ]:
# 사용자가 안본 영화 중 예측 평점 높은 10개 추천
def get_recomendation(user_id):
    user_mov = df[df['userId'] == user_id]
    user_mov_pred =  df_pred_all_parallel[ df_pred_all_parallel['userId'] == user_id]
    user_mov = pd.merge(user_mov, user_mov_pred, on=['userId', 'title'], how='right')

    # 사용자가 아직 안 본 영화
    movie_candidate = user_mov[user_mov['movieId'].isnull()]
    movie_candidate = movie_candidate.sort_values(by='pred_rating', ascending=False)[:10]

    return movie_candidate['title'].tolist()

In [ ]:
get_recomendation(1)

['Win Win',
 'Charlie Bartlett',
 'Animal Crackers',
 'On the Town',
 'Night at the Opera, A',
 'Day & Night',
 'Ex, The',
 'Victory (a.k.a. Escape to Victory)',
 'Face in the Crowd, A',
 'Place Beyond the Pines, The']